In [1]:
from google.colab import drive
drive.mount('/content/drive')
words = open('/content/drive/MyDrive/Colab Notebooks/names.txt', 'r').read().splitlines()

Mounted at /content/drive


In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [12]:
block_size = 3
X,Y = [],[]

for w in words[:5]:
  context = [0]*block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
... ----> a
..a ----> v
.av ----> a
ava ----> .
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [13]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [14]:
C = torch.randn((27,2))
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [15]:
C

tensor([[-0.2953,  0.7190],
        [ 0.4081, -0.3067],
        [ 0.4054, -1.1298],
        [ 0.8978,  0.4153],
        [-0.3254,  0.1050],
        [-0.1428, -1.1788],
        [ 0.7575, -0.1371],
        [ 1.1944, -0.0213],
        [ 0.1666,  0.9324],
        [-1.0958,  0.8008],
        [ 0.7362,  0.1868],
        [-0.4179,  0.0342],
        [-2.1461, -0.3129],
        [-0.0618,  0.3168],
        [ 1.1673,  0.0432],
        [ 0.0504,  0.8522],
        [ 0.6563,  0.4882],
        [ 1.6282,  0.7667],
        [ 1.1652,  1.1912],
        [ 0.6526, -0.5510],
        [ 1.3525,  0.9200],
        [ 0.5500,  0.1042],
        [ 1.0848, -1.1072],
        [ 1.6735,  0.5826],
        [ 0.4377,  0.1630],
        [-0.8812,  0.1292],
        [ 0.7574, -0.9536]])

In [16]:
emb

tensor([[[-0.2953,  0.7190],
         [-0.2953,  0.7190],
         [-0.2953,  0.7190]],

        [[-0.2953,  0.7190],
         [-0.2953,  0.7190],
         [-0.1428, -1.1788]],

        [[-0.2953,  0.7190],
         [-0.1428, -1.1788],
         [-0.0618,  0.3168]],

        [[-0.1428, -1.1788],
         [-0.0618,  0.3168],
         [-0.0618,  0.3168]],

        [[-0.0618,  0.3168],
         [-0.0618,  0.3168],
         [ 0.4081, -0.3067]],

        [[-0.2953,  0.7190],
         [-0.2953,  0.7190],
         [-0.2953,  0.7190]],

        [[-0.2953,  0.7190],
         [-0.2953,  0.7190],
         [ 0.0504,  0.8522]],

        [[-0.2953,  0.7190],
         [ 0.0504,  0.8522],
         [-2.1461, -0.3129]],

        [[ 0.0504,  0.8522],
         [-2.1461, -0.3129],
         [-1.0958,  0.8008]],

        [[-2.1461, -0.3129],
         [-1.0958,  0.8008],
         [ 1.0848, -1.1072]],

        [[-1.0958,  0.8008],
         [ 1.0848, -1.1072],
         [-1.0958,  0.8008]],

        [[ 1.0848, -1

In [19]:
emb.view(-1,6)

tensor([[-0.2953,  0.7190, -0.2953,  0.7190, -0.2953,  0.7190],
        [-0.2953,  0.7190, -0.2953,  0.7190, -0.1428, -1.1788],
        [-0.2953,  0.7190, -0.1428, -1.1788, -0.0618,  0.3168],
        [-0.1428, -1.1788, -0.0618,  0.3168, -0.0618,  0.3168],
        [-0.0618,  0.3168, -0.0618,  0.3168,  0.4081, -0.3067],
        [-0.2953,  0.7190, -0.2953,  0.7190, -0.2953,  0.7190],
        [-0.2953,  0.7190, -0.2953,  0.7190,  0.0504,  0.8522],
        [-0.2953,  0.7190,  0.0504,  0.8522, -2.1461, -0.3129],
        [ 0.0504,  0.8522, -2.1461, -0.3129, -1.0958,  0.8008],
        [-2.1461, -0.3129, -1.0958,  0.8008,  1.0848, -1.1072],
        [-1.0958,  0.8008,  1.0848, -1.1072, -1.0958,  0.8008],
        [ 1.0848, -1.1072, -1.0958,  0.8008,  0.4081, -0.3067],
        [-0.2953,  0.7190, -0.2953,  0.7190, -0.2953,  0.7190],
        [-0.2953,  0.7190, -0.2953,  0.7190,  0.4081, -0.3067],
        [-0.2953,  0.7190,  0.4081, -0.3067,  1.0848, -1.1072],
        [ 0.4081, -0.3067,  1.0848, -1.1

In [ ]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
h.shape

torch.Size([228146, 100])

In [ ]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts/counts.sum(1, keepdims=True)
prob.shape

torch.Size([228146, 27])

In [ ]:
#cleaned up
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100))
b1 = torch.randn(100)
W2 = torch.randn((100,27))
b2 = torch.randn(27)
parameters = [C,W1,b1,W2,b2]

In [ ]:
sum(p.nelement() for p in parameters)

3481

In [ ]:
for p in parameters:
  p.requires_grad = True

In [ ]:
lre = torch.linspace(-3,0,1000)
lrs = 10**lre

In [ ]:
lri = []
lossi = []

for i in range(1000):
  #minibatch construct
  ix = torch.randint(0, X.shape[0], (32,))

  #forward
  emb = C[X[ix]]
  h = torch.tanh(emb.view(-1,6) @ W1 + b1)
  logits = h @ W2 + b2
  # counts = logits.exp()
  # prob = counts/counts.sum(1, keepdims=True)
  # loss = -prob[torch.arange(32), Y].log().mean()
  loss = F.cross_entropy(logits, Y[ix])

  #backward
  for p in parameters:
    p.grad=None
  loss.backward()

  #update
  #lr = lrs[i]
  for p in parameters:
    p.data += -0.1 * p.grad

  #track stats
  # lri.append(lr)
  # lossi.append(loss.item())

print(loss.item())

2.449061632156372


In [ ]:
#plt.plot(lri,lossi)

In [ ]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss

tensor(2.5438, grad_fn=<NllLossBackward0>)